# Comp 472 Mini-Project 1 Fall 2022

## Instructions

   **Goal**: Implement A* search and test different heuristics for the 8-puzzle using the goal state from the book: 
    1 2 3 
    8 _ 4 
    7 6 5

   **Input**: a state is a board configuration in form of a list. The board configuration of the goal state would be represented as: (1 2 3 8 B 4 7 6 5). Remember, that not all initial configurations can be transformed into this goal configuration, take care with your test examples to be solvable.

   **Output**: A list of successive states, beginning with the start state and ending with the goal state, and including all states required to transform the start state into the goal state. For admissible heuristics this path should be optimal.
    
### Description:    
    
- [X] Implement a general search as described on the slides, with OPEN and CLOSED lists. Implement different OPEN list orderings (for best-first, depth-first, Best-First, and A* (2.5pts, Attrib. 5)
- [X] Implement a successor state generator for the 8-puzzle (2.5pts, Attrib 5)
- [X] Implement Hamming distance, Manhatten distance, Permutation Inversion heuristics for the 8-puzzle (2pts, Attrib. 5)
- [X] Compare the length of the search path and the optimality of the solution found for Best-First and A* for the three admissible heuristics mentioned above plus an inadmissible heuristic (you may use the one from Assignment 1) (1pt, Attrib 1,4)
- [X] On March 10, 2022, I will post an initial configuration that you have to solve with your code. Report on your solution in your report. (1pt, Attrib 5,4)
- [X] Provide ample and clear documentation in-line in your code and in a report. The report should be an electronic document in .pdf that shows how the different heuristics perform on different test runs for both, Best-First and A* (in tabular form) with some analysis and explanation. Explain what happens for the inadmissible heuristic. (Make sure the marker knows which one is the inadmissible one in the code, too.) Explain how many experiments you have run to compile the table (1pt, Attrib 1,4)

### Challenge:

* Challenge:

    * Input: (2 8 3 1 6 4 7 B 5)
    * Goal: (1 2 3 8 B 4 7 6 5)


* Greater challenge (to showcase your heuristics):

    * Start: (5 1 4 7 B 6 3 8 2)
    * Goal: (1 2 3 8 B 4 7 6 5)

In [118]:
''' The following source code implements search heuristics to solve the 8-puzzle problem. 
Class Node is used to represent nodes in a search tree, and class Eightpuzzle is used to represent the 8-puzzle problem. 
Implemented Searches: breadth-first, depth-first, Best-First, and A*
Implemented Heuristics: Hamming distance, Manhatten distance, Permutation Inversion heuristics, inadmissible heuristic
Reference used: Stuart Russell and Peter Norvig, Artificial Intelligence: A Modern Approach, 4th edition, Prentice Hall.'''

import sys
import math
from collections import deque

 class Node adapted from [Stuart Russell and Peter Norvig, Artificial Intelligence: A Modern Approach, 4th edition, Prentice Hall](https://github.com/aimacode/aima-python/blob/master/search.py)

In [171]:
class Node:

    # constructor
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost

    # successor state generator function
    def expand(self, problem):
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    # successor state generator helper function 
    def child_node(self, problem, action):
        next_state = problem.result(self.state, action)
        next_node = Node(next_state, self, action, problem.path_cost(self.path_cost, self.state, action, next_state))
        return next_node

    # list of actions from root to this node
    def solution(self):
        return [node.action for node in self.path()[1:]]

    # list of nodes from root to this note
    def path(self):
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

    # node represented in list form
    def __repr__(self):
        return "{}".format(self.state)

In [172]:
class EightPuzzle:
    
    # constructor
    def __init__(self, initial, goal=(1, 2, 3, 8, 'B', 4, 7, 6, 5)):
        self.initial = initial
        self.goal = goal

    # return list of possible indices for "B" square
    def actions(self, state):
        indices = ((1, 3),    (0, 2, 4),    (1, 5),
                 (0, 4, 6), (1, 3, 5, 7), (2, 4, 8),
                 (3, 7),    (4, 6, 8),    (7, 5))
        blank_square = state.index('B')
        return indices[blank_square]

    # moves "B" square
    def result(self, state, action):
        new_state = list(state)
        blank_square = state.index('B')
        new_state[action], new_state[blank_square] = new_state[blank_square], new_state[action]
        return tuple(new_state)

    # returns true if state is goal state
    def is_goal_state(self, state):
        return state == self.goal
    
    def path_cost(self, path_cost, state, action, next_state):
        return path_cost + 1
    
    # returns hamming distance value for a given state 
    def hamming_distance(self, node):
        sum = 0
        for i in range(1,9): # only considers numbered tiles
            if node.state.index(i) != self.goal.index(i):
                sum += 1
        return sum
    
    # returns manhattan distance value for a given state 
    def manhattan_distance(self, node):
        index = [(0,0), (0,1), (0,2), (1,0), (1,1), (1,2), (2,0), (2,1), (2,2)]
        index_goal = dict(zip(self.goal, index))
        index_state = dict(zip(node.state, index))
              
        sum = 0
        for i in range(1,9): # only considers numbered tiles
            x1, y1 = index_state[i]
            x2, y2 = index_goal[i]
            sum += abs(x2-x1) + abs(y2-y1)
        return sum
    
    # returns permutation inversions value for a given state
    def permutation_inversions(self, node):
        goal = self.goal
        state = node.state

        # only considers numbered tiles
        goal = list(goal)
        goal.remove('B')
        state = list(state)
        state.remove('B')

        sum = 0
        for i in range(len(state)):
            value = state[i]
            goal_index = goal.index(value)
            list1 = goal[:goal_index]
            list2 = state[i+1:]
            list3 = [value for value in list1 if value in list2]
            sum += len(list3)
        return sum
    
    # returns inadmissible heuristic value for a given state
    def inadmissible_heuristic(self, node):
        return (self.hamming_distance(node) + 2*self.manhattan_distance(node))

In [173]:
challenge = EightPuzzle((2, 8, 3, 1, 6, 4, 7, 'B', 5))

In [174]:
greater_challenge = EightPuzzle((5, 1, 4, 7, 'B', 6, 3, 8, 2))

## Testing Heuristic Functions on challenge puzzles

### EightPuzzle((2, 8, 3, 1, 6, 4, 7, 'B', 5))

In [175]:
challenge.hamming_distance((Node(challenge.initial)))

4

In [176]:
challenge.manhattan_distance((Node(challenge.initial)))

5

In [177]:
challenge.permutation_inversions((Node(challenge.initial)))

6

In [178]:
challenge.inadmissible_heuristic((Node(challenge.initial)))

14

### EightPuzzle((5, 1, 4, 7, 'B', 6, 3, 8, 2))

In [179]:
greater_challenge.hamming_distance((Node(greater_challenge.initial)))

8

In [180]:
greater_challenge.manhattan_distance((Node(greater_challenge.initial)))

18

In [181]:
greater_challenge.permutation_inversions((Node(greater_challenge.initial)))

18

In [182]:
greater_challenge.inadmissible_heuristic((Node(greater_challenge.initial)))

44

## Testing  successor state generator on challenge puzzles


### EightPuzzle((2, 8, 3, 1, 6, 4, 7, 'B', 5))

In [183]:
children_challenge = Node(challenge.initial).expand(challenge)

In [184]:
children_challenge

[(2, 8, 3, 1, 'B', 4, 7, 6, 5),
 (2, 8, 3, 1, 6, 4, 'B', 7, 5),
 (2, 8, 3, 1, 6, 4, 7, 5, 'B')]

### EightPuzzle((5, 1, 4, 7, 'B', 6, 3, 8, 2))

In [185]:
children_greater_challenge = Node(greater_challenge.initial).expand(greater_challenge)

In [186]:
children_greater_challenge

[(5, 'B', 4, 7, 1, 6, 3, 8, 2),
 (5, 1, 4, 'B', 7, 6, 3, 8, 2),
 (5, 1, 4, 7, 6, 'B', 3, 8, 2),
 (5, 1, 4, 7, 8, 6, 3, 'B', 2)]

## Search Functions

### 1. Breadth First Search

In [187]:
def breadth_first_search(eightpuzzle):

    open_list = deque([(Node(eightpuzzle.initial))])
    closed_list = set()
    
    while open_list:
        node = open_list.popleft()
        if eightpuzzle.is_goal_state(node.state):
            return node
        closed_list.add(node.state)
        
        for child in node.expand(eightpuzzle):
            if child.state not in closed_list and child not in open_list:
                open_list.append(child)
    return None

### 2. Depth First Search

In [188]:
def depth_first_search(eightpuzzle):

    open_list = deque([(Node(eightpuzzle.initial))])
    closed_list = set()
    
    while open_list:
        node = open_list.popleft()
        if eightpuzzle.is_goal_state(node.state):
            return node
        closed_list.add(node.state)
        
        for child in node.expand(eightpuzzle):
            if child.state not in closed_list and child not in open_list:
                open_list.appendleft(child)
    return None

### 3. Best First Search

In [189]:
def best_first_search(eightpuzzle, value):
    
    node = Node(eightpuzzle.initial)
    open_list = [node]
    closed_list = set()
    
    while open_list:
        open_list.sort(key=value, reverse=True)
        node = EightPuzzle(open_list.pop()).initial
        if eightpuzzle.is_goal_state(node.state):
            return node
        closed_list.add(node.state)
        
        for child in node.expand(eightpuzzle):
            if child.state not in closed_list and child not in open_list:
                open_list.append(child)
    return None

### 4. A* Search

In [190]:
def a_star_search(eightpuzzle, value):
    
    node = Node(eightpuzzle.initial)
    open_list = [node]
    closed_list = set()
    
    while open_list:
        open_list.sort(key=value , reverse=True)
        node = EightPuzzle(open_list.pop()).initial
        if eightpuzzle.is_goal_state(node.state):
            return node
        closed_list.add(node.state)
        
        for child in node.expand(eightpuzzle):
            if child.state not in closed_list and child not in open_list:
                open_list.append(child)
    return None

## Testing  search functions with challenge puzzles


### EightPuzzle((2, 8, 3, 1, 6, 4, 7, 'B', 5))

### 1. Breadth First Search

In [191]:
bfs_challenge_path = breadth_first_search(challenge).path()
print(bfs_challenge_path)

[(2, 8, 3, 1, 6, 4, 7, 'B', 5), (2, 8, 3, 1, 'B', 4, 7, 6, 5), (2, 'B', 3, 1, 8, 4, 7, 6, 5), ('B', 2, 3, 1, 8, 4, 7, 6, 5), (1, 2, 3, 'B', 8, 4, 7, 6, 5), (1, 2, 3, 8, 'B', 4, 7, 6, 5)]


### 2. Depth First Search

In [192]:
dfs_challenge_path = depth_first_search(challenge).path()
print(dfs_challenge_path)

[(2, 8, 3, 1, 6, 4, 7, 'B', 5), (2, 8, 3, 1, 6, 4, 7, 5, 'B'), (2, 8, 3, 1, 6, 'B', 7, 5, 4), (2, 8, 3, 1, 'B', 6, 7, 5, 4), (2, 8, 3, 1, 5, 6, 7, 'B', 4), (2, 8, 3, 1, 5, 6, 7, 4, 'B'), (2, 8, 3, 1, 5, 'B', 7, 4, 6), (2, 8, 3, 1, 'B', 5, 7, 4, 6), (2, 8, 3, 1, 4, 5, 7, 'B', 6), (2, 8, 3, 1, 4, 5, 7, 6, 'B'), (2, 8, 3, 1, 4, 'B', 7, 6, 5), (2, 8, 3, 1, 'B', 4, 7, 6, 5), (2, 8, 3, 'B', 1, 4, 7, 6, 5), (2, 8, 3, 7, 1, 4, 'B', 6, 5), (2, 8, 3, 7, 1, 4, 6, 'B', 5), (2, 8, 3, 7, 1, 4, 6, 5, 'B'), (2, 8, 3, 7, 1, 'B', 6, 5, 4), (2, 8, 3, 7, 'B', 1, 6, 5, 4), (2, 8, 3, 7, 5, 1, 6, 'B', 4), (2, 8, 3, 7, 5, 1, 6, 4, 'B'), (2, 8, 3, 7, 5, 'B', 6, 4, 1), (2, 8, 3, 7, 'B', 5, 6, 4, 1), (2, 8, 3, 7, 4, 5, 6, 'B', 1), (2, 8, 3, 7, 4, 5, 6, 1, 'B'), (2, 8, 3, 7, 4, 'B', 6, 1, 5), (2, 8, 3, 7, 'B', 4, 6, 1, 5), (2, 8, 3, 'B', 7, 4, 6, 1, 5), (2, 8, 3, 6, 7, 4, 'B', 1, 5), (2, 8, 3, 6, 7, 4, 1, 'B', 5), (2, 8, 3, 6, 7, 4, 1, 5, 'B'), (2, 8, 3, 6, 7, 'B', 1, 5, 4), (2, 8, 3, 6, 'B', 7, 1, 5, 4), (2, 8, 

### 3. Best First Search

In [246]:
bf_challenge_hamming_path = best_first_search(challenge, lambda x: challenge.hamming_distance(x)).path()
print(bf_challenge_hamming_path)

[(2, 8, 3, 1, 6, 4, 7, 'B', 5), (2, 8, 3, 1, 'B', 4, 7, 6, 5), (2, 8, 3, 'B', 1, 4, 7, 6, 5), ('B', 8, 3, 2, 1, 4, 7, 6, 5), (8, 'B', 3, 2, 1, 4, 7, 6, 5), (8, 1, 3, 2, 'B', 4, 7, 6, 5), (8, 1, 3, 'B', 2, 4, 7, 6, 5), ('B', 1, 3, 8, 2, 4, 7, 6, 5), (1, 'B', 3, 8, 2, 4, 7, 6, 5), (1, 2, 3, 8, 'B', 4, 7, 6, 5)]


In [247]:
bf_challenge_manhattan_path = best_first_search(challenge, lambda x: challenge.manhattan_distance(x)).path()
print(bf_challenge_manhattan_path)

[(2, 8, 3, 1, 6, 4, 7, 'B', 5), (2, 8, 3, 1, 'B', 4, 7, 6, 5), (2, 'B', 3, 1, 8, 4, 7, 6, 5), ('B', 2, 3, 1, 8, 4, 7, 6, 5), (1, 2, 3, 'B', 8, 4, 7, 6, 5), (1, 2, 3, 8, 'B', 4, 7, 6, 5)]


In [248]:
bf_challenge_permutation_inversions_path = best_first_search(challenge, lambda x: challenge.permutation_inversions(x)).path()
print(bf_challenge_permutation_inversions_path)

[(2, 8, 3, 1, 6, 4, 7, 'B', 5), (2, 8, 3, 1, 'B', 4, 7, 6, 5), (2, 'B', 3, 1, 8, 4, 7, 6, 5), ('B', 2, 3, 1, 8, 4, 7, 6, 5), (1, 2, 3, 'B', 8, 4, 7, 6, 5), (1, 2, 3, 8, 'B', 4, 7, 6, 5)]


In [249]:
bf_challenge_inadmissible_heuristic_path = best_first_search(challenge, lambda x: challenge.inadmissible_heuristic(x)).path()
print(bf_challenge_inadmissible_heuristic_path)

[(2, 8, 3, 1, 6, 4, 7, 'B', 5), (2, 8, 3, 1, 'B', 4, 7, 6, 5), (2, 'B', 3, 1, 8, 4, 7, 6, 5), ('B', 2, 3, 1, 8, 4, 7, 6, 5), (1, 2, 3, 'B', 8, 4, 7, 6, 5), (1, 2, 3, 8, 'B', 4, 7, 6, 5)]


### 4. A* Search

In [250]:
a_star_challenge_hamming_path = a_star_search(challenge, lambda x: challenge.hamming_distance(x) + x.path_cost).path()
print(bf_challenge_hamming_path)

[(2, 8, 3, 1, 6, 4, 7, 'B', 5), (2, 8, 3, 1, 'B', 4, 7, 6, 5), (2, 8, 3, 'B', 1, 4, 7, 6, 5), ('B', 8, 3, 2, 1, 4, 7, 6, 5), (8, 'B', 3, 2, 1, 4, 7, 6, 5), (8, 1, 3, 2, 'B', 4, 7, 6, 5), (8, 1, 3, 'B', 2, 4, 7, 6, 5), ('B', 1, 3, 8, 2, 4, 7, 6, 5), (1, 'B', 3, 8, 2, 4, 7, 6, 5), (1, 2, 3, 8, 'B', 4, 7, 6, 5)]


In [251]:
a_star_challenge_manhattan_path = a_star_search(challenge, lambda x: challenge.manhattan_distance(x) + x.path_cost).path()
print(bf_challenge_manhattan_path)

[(2, 8, 3, 1, 6, 4, 7, 'B', 5), (2, 8, 3, 1, 'B', 4, 7, 6, 5), (2, 'B', 3, 1, 8, 4, 7, 6, 5), ('B', 2, 3, 1, 8, 4, 7, 6, 5), (1, 2, 3, 'B', 8, 4, 7, 6, 5), (1, 2, 3, 8, 'B', 4, 7, 6, 5)]


In [252]:
a_star_challenge_permutation_inversions_path = a_star_search(challenge, lambda x: challenge.permutation_inversions(x) + x.path_cost).path()
print(bf_challenge_permutation_inversions_path)

[(2, 8, 3, 1, 6, 4, 7, 'B', 5), (2, 8, 3, 1, 'B', 4, 7, 6, 5), (2, 'B', 3, 1, 8, 4, 7, 6, 5), ('B', 2, 3, 1, 8, 4, 7, 6, 5), (1, 2, 3, 'B', 8, 4, 7, 6, 5), (1, 2, 3, 8, 'B', 4, 7, 6, 5)]


In [253]:
a_star_challenge_inadmissible_heuristic_path = a_star_search(challenge, lambda x: challenge.inadmissible_heuristic(x)).path()
print(bf_challenge_inadmissible_heuristic_path)

[(2, 8, 3, 1, 6, 4, 7, 'B', 5), (2, 8, 3, 1, 'B', 4, 7, 6, 5), (2, 'B', 3, 1, 8, 4, 7, 6, 5), ('B', 2, 3, 1, 8, 4, 7, 6, 5), (1, 2, 3, 'B', 8, 4, 7, 6, 5), (1, 2, 3, 8, 'B', 4, 7, 6, 5)]


### EightPuzzle((5, 1, 4, 7, 'B', 6, 3, 8, 2))

### 1. Breadth First Search

In [201]:
bfs_greater_challenge_path = breadth_first_search(greater_challenge).path()
print(bfs_greater_challenge_path)

[(5, 1, 4, 7, 'B', 6, 3, 8, 2), (5, 1, 4, 'B', 7, 6, 3, 8, 2), (5, 1, 4, 3, 7, 6, 'B', 8, 2), (5, 1, 4, 3, 7, 6, 8, 'B', 2), (5, 1, 4, 3, 'B', 6, 8, 7, 2), (5, 1, 4, 'B', 3, 6, 8, 7, 2), ('B', 1, 4, 5, 3, 6, 8, 7, 2), (1, 'B', 4, 5, 3, 6, 8, 7, 2), (1, 3, 4, 5, 'B', 6, 8, 7, 2), (1, 3, 4, 'B', 5, 6, 8, 7, 2), (1, 3, 4, 8, 5, 6, 'B', 7, 2), (1, 3, 4, 8, 5, 6, 7, 'B', 2), (1, 3, 4, 8, 'B', 6, 7, 5, 2), (1, 3, 4, 8, 6, 'B', 7, 5, 2), (1, 3, 4, 8, 6, 2, 7, 5, 'B'), (1, 3, 4, 8, 6, 2, 7, 'B', 5), (1, 3, 4, 8, 'B', 2, 7, 6, 5), (1, 3, 4, 8, 2, 'B', 7, 6, 5), (1, 3, 'B', 8, 2, 4, 7, 6, 5), (1, 'B', 3, 8, 2, 4, 7, 6, 5), (1, 2, 3, 8, 'B', 4, 7, 6, 5)]


### 2. Depth First Search

In [202]:
dfs_greater_challenge_path = depth_first_search(greater_challenge).path()
print(dfs_greater_challenge_path)

[(5, 1, 4, 7, 'B', 6, 3, 8, 2), (5, 1, 4, 7, 8, 6, 3, 'B', 2), (5, 1, 4, 7, 8, 6, 3, 2, 'B'), (5, 1, 4, 7, 8, 'B', 3, 2, 6), (5, 1, 4, 7, 'B', 8, 3, 2, 6), (5, 1, 4, 7, 2, 8, 3, 'B', 6), (5, 1, 4, 7, 2, 8, 3, 6, 'B'), (5, 1, 4, 7, 2, 'B', 3, 6, 8), (5, 1, 4, 7, 'B', 2, 3, 6, 8), (5, 1, 4, 7, 6, 2, 3, 'B', 8), (5, 1, 4, 7, 6, 2, 3, 8, 'B'), (5, 1, 4, 7, 6, 'B', 3, 8, 2), (5, 1, 'B', 7, 6, 4, 3, 8, 2), (5, 'B', 1, 7, 6, 4, 3, 8, 2), (5, 6, 1, 7, 'B', 4, 3, 8, 2), (5, 6, 1, 7, 8, 4, 3, 'B', 2), (5, 6, 1, 7, 8, 4, 3, 2, 'B'), (5, 6, 1, 7, 8, 'B', 3, 2, 4), (5, 6, 1, 7, 'B', 8, 3, 2, 4), (5, 6, 1, 7, 2, 8, 3, 'B', 4), (5, 6, 1, 7, 2, 8, 3, 4, 'B'), (5, 6, 1, 7, 2, 'B', 3, 4, 8), (5, 6, 1, 7, 'B', 2, 3, 4, 8), (5, 6, 1, 7, 4, 2, 3, 'B', 8), (5, 6, 1, 7, 4, 2, 3, 8, 'B'), (5, 6, 1, 7, 4, 'B', 3, 8, 2), (5, 6, 'B', 7, 4, 1, 3, 8, 2), (5, 'B', 6, 7, 4, 1, 3, 8, 2), (5, 4, 6, 7, 'B', 1, 3, 8, 2), (5, 4, 6, 7, 8, 1, 3, 'B', 2), (5, 4, 6, 7, 8, 1, 3, 2, 'B'), (5, 4, 6, 7, 8, 'B', 3, 2, 1), (5, 4, 

### 3. Best First Search

In [203]:
bf_greater_challenge_hamming_path = best_first_search(greater_challenge, lambda x: greater_challenge.hamming_distance(x)).path()
print(bf_greater_challenge_hamming_path)

[(5, 1, 4, 7, 'B', 6, 3, 8, 2), (5, 1, 4, 7, 8, 6, 3, 'B', 2), (5, 1, 4, 7, 8, 6, 3, 2, 'B'), (5, 1, 4, 7, 8, 'B', 3, 2, 6), (5, 1, 'B', 7, 8, 4, 3, 2, 6), (5, 'B', 1, 7, 8, 4, 3, 2, 6), (5, 8, 1, 7, 'B', 4, 3, 2, 6), (5, 8, 1, 7, 2, 4, 3, 'B', 6), (5, 8, 1, 7, 2, 4, 3, 6, 'B'), (5, 8, 1, 7, 2, 'B', 3, 6, 4), (5, 8, 1, 7, 'B', 2, 3, 6, 4), (5, 8, 1, 'B', 7, 2, 3, 6, 4), ('B', 8, 1, 5, 7, 2, 3, 6, 4), (8, 'B', 1, 5, 7, 2, 3, 6, 4), (8, 7, 1, 5, 'B', 2, 3, 6, 4), (8, 7, 1, 5, 2, 'B', 3, 6, 4), (8, 7, 1, 5, 2, 4, 3, 6, 'B'), (8, 7, 1, 5, 2, 4, 3, 'B', 6), (8, 7, 1, 5, 2, 4, 'B', 3, 6), (8, 7, 1, 'B', 2, 4, 5, 3, 6), ('B', 7, 1, 8, 2, 4, 5, 3, 6), (7, 'B', 1, 8, 2, 4, 5, 3, 6), (7, 2, 1, 8, 'B', 4, 5, 3, 6), (7, 2, 1, 8, 3, 4, 5, 'B', 6), (7, 2, 1, 8, 3, 4, 'B', 5, 6), (7, 2, 1, 'B', 3, 4, 8, 5, 6), (7, 2, 1, 3, 'B', 4, 8, 5, 6), (7, 2, 1, 3, 5, 4, 8, 'B', 6), (7, 2, 1, 3, 5, 4, 8, 6, 'B'), (7, 2, 1, 3, 5, 'B', 8, 6, 4), (7, 2, 1, 3, 'B', 5, 8, 6, 4), (7, 2, 1, 'B', 3, 5, 8, 6, 4), (7, 2, 

In [204]:
bf_greater_challenge_manhattan_path = best_first_search(greater_challenge, lambda x: greater_challenge.manhattan_distance(x)).path()
print(bf_greater_challenge_manhattan_path)

[(5, 1, 4, 7, 'B', 6, 3, 8, 2), (5, 1, 4, 7, 8, 6, 3, 'B', 2), (5, 1, 4, 7, 8, 6, 3, 2, 'B'), (5, 1, 4, 7, 8, 'B', 3, 2, 6), (5, 1, 'B', 7, 8, 4, 3, 2, 6), (5, 'B', 1, 7, 8, 4, 3, 2, 6), ('B', 5, 1, 7, 8, 4, 3, 2, 6), (7, 5, 1, 'B', 8, 4, 3, 2, 6), (7, 5, 1, 8, 'B', 4, 3, 2, 6), (7, 5, 1, 8, 2, 4, 3, 'B', 6), (7, 5, 1, 8, 2, 4, 'B', 3, 6), (7, 5, 1, 'B', 2, 4, 8, 3, 6), ('B', 5, 1, 7, 2, 4, 8, 3, 6), (5, 'B', 1, 7, 2, 4, 8, 3, 6), (5, 2, 1, 7, 'B', 4, 8, 3, 6), (5, 2, 1, 7, 3, 4, 8, 'B', 6), (5, 2, 1, 7, 3, 4, 8, 6, 'B'), (5, 2, 1, 7, 3, 'B', 8, 6, 4), (5, 2, 1, 7, 'B', 3, 8, 6, 4), (5, 2, 1, 'B', 7, 3, 8, 6, 4), ('B', 2, 1, 5, 7, 3, 8, 6, 4), (2, 'B', 1, 5, 7, 3, 8, 6, 4), (2, 1, 'B', 5, 7, 3, 8, 6, 4), (2, 1, 3, 5, 7, 'B', 8, 6, 4), (2, 1, 3, 5, 7, 4, 8, 6, 'B'), (2, 1, 3, 5, 7, 4, 8, 'B', 6), (2, 1, 3, 5, 'B', 4, 8, 7, 6), (2, 1, 3, 'B', 5, 4, 8, 7, 6), (2, 1, 3, 8, 5, 4, 'B', 7, 6), (2, 1, 3, 8, 5, 4, 7, 'B', 6), (2, 1, 3, 8, 'B', 4, 7, 5, 6), (2, 'B', 3, 8, 1, 4, 7, 5, 6), ('B', 2

In [205]:
bf_greater_challenge_permutation_inversions_path = best_first_search(greater_challenge, lambda x: greater_challenge.permutation_inversions(x)).path()
print(bf_greater_challenge_permutation_inversions_path)

[(5, 1, 4, 7, 'B', 6, 3, 8, 2), (5, 1, 4, 7, 8, 6, 3, 'B', 2), (5, 1, 4, 7, 8, 6, 3, 2, 'B'), (5, 1, 4, 7, 8, 'B', 3, 2, 6), (5, 1, 4, 7, 'B', 8, 3, 2, 6), (5, 1, 4, 7, 2, 8, 3, 'B', 6), (5, 1, 4, 7, 2, 8, 3, 6, 'B'), (5, 1, 4, 7, 2, 'B', 3, 6, 8), (5, 1, 4, 7, 'B', 2, 3, 6, 8), (5, 1, 4, 'B', 7, 2, 3, 6, 8), ('B', 1, 4, 5, 7, 2, 3, 6, 8), (1, 'B', 4, 5, 7, 2, 3, 6, 8), (1, 7, 4, 5, 'B', 2, 3, 6, 8), (1, 7, 4, 5, 2, 'B', 3, 6, 8), (1, 7, 4, 5, 2, 8, 3, 6, 'B'), (1, 7, 4, 5, 2, 8, 3, 'B', 6), (1, 7, 4, 5, 2, 8, 'B', 3, 6), (1, 7, 4, 'B', 2, 8, 5, 3, 6), (1, 7, 4, 2, 'B', 8, 5, 3, 6), (1, 'B', 4, 2, 7, 8, 5, 3, 6), (1, 4, 'B', 2, 7, 8, 5, 3, 6), (1, 4, 8, 2, 7, 'B', 5, 3, 6), (1, 4, 8, 2, 'B', 7, 5, 3, 6), (1, 'B', 8, 2, 4, 7, 5, 3, 6), ('B', 1, 8, 2, 4, 7, 5, 3, 6), (2, 1, 8, 'B', 4, 7, 5, 3, 6), (2, 1, 8, 4, 'B', 7, 5, 3, 6), (2, 1, 8, 4, 3, 7, 5, 'B', 6), (2, 1, 8, 4, 3, 7, 'B', 5, 6), (2, 1, 8, 'B', 3, 7, 4, 5, 6), (2, 1, 8, 3, 'B', 7, 4, 5, 6), (2, 1, 8, 3, 7, 'B', 4, 5, 6), (2, 1, 

In [206]:
bf_greater_challenge_inadmissible_heuristic_path = best_first_search(greater_challenge, lambda x: greater_challenge.inadmissible_heuristic(x)).path()
print(bf_greater_challenge_inadmissible_heuristic_path)

[(5, 1, 4, 7, 'B', 6, 3, 8, 2), (5, 1, 4, 7, 8, 6, 3, 'B', 2), (5, 1, 4, 7, 8, 6, 3, 2, 'B'), (5, 1, 4, 7, 8, 'B', 3, 2, 6), (5, 1, 'B', 7, 8, 4, 3, 2, 6), (5, 'B', 1, 7, 8, 4, 3, 2, 6), ('B', 5, 1, 7, 8, 4, 3, 2, 6), (7, 5, 1, 'B', 8, 4, 3, 2, 6), (7, 5, 1, 8, 'B', 4, 3, 2, 6), (7, 'B', 1, 8, 5, 4, 3, 2, 6), (7, 1, 'B', 8, 5, 4, 3, 2, 6), (7, 1, 4, 8, 5, 'B', 3, 2, 6), (7, 1, 4, 8, 'B', 5, 3, 2, 6), (7, 1, 4, 8, 2, 5, 3, 'B', 6), (7, 1, 4, 8, 2, 5, 3, 6, 'B'), (7, 1, 4, 8, 2, 'B', 3, 6, 5), (7, 1, 'B', 8, 2, 4, 3, 6, 5), (7, 'B', 1, 8, 2, 4, 3, 6, 5), (7, 2, 1, 8, 'B', 4, 3, 6, 5), (7, 2, 1, 8, 6, 4, 3, 'B', 5), (7, 2, 1, 8, 6, 4, 'B', 3, 5), (7, 2, 1, 'B', 6, 4, 8, 3, 5), (7, 2, 1, 6, 'B', 4, 8, 3, 5), (7, 2, 1, 6, 3, 4, 8, 'B', 5), (7, 2, 1, 6, 3, 4, 'B', 8, 5), (7, 2, 1, 'B', 3, 4, 6, 8, 5), ('B', 2, 1, 7, 3, 4, 6, 8, 5), (2, 'B', 1, 7, 3, 4, 6, 8, 5), (2, 3, 1, 7, 'B', 4, 6, 8, 5), (2, 3, 1, 7, 8, 4, 6, 'B', 5), (2, 3, 1, 7, 8, 4, 'B', 6, 5), (2, 3, 1, 'B', 8, 4, 7, 6, 5), (2, 3, 

### 4. A* Search

In [207]:
a_star_greater_challenge_hamming_path = a_star_search(greater_challenge, lambda x: greater_challenge.hamming_distance(x) + x.path_cost).path()
print(a_star_greater_challenge_hamming_path)

[(5, 1, 4, 7, 'B', 6, 3, 8, 2), (5, 1, 4, 'B', 7, 6, 3, 8, 2), (5, 1, 4, 3, 7, 6, 'B', 8, 2), (5, 1, 4, 3, 7, 6, 8, 'B', 2), (5, 1, 4, 3, 'B', 6, 8, 7, 2), (5, 1, 4, 'B', 3, 6, 8, 7, 2), ('B', 1, 4, 5, 3, 6, 8, 7, 2), (1, 'B', 4, 5, 3, 6, 8, 7, 2), (1, 3, 4, 5, 'B', 6, 8, 7, 2), (1, 3, 4, 'B', 5, 6, 8, 7, 2), (1, 3, 4, 8, 5, 6, 'B', 7, 2), (1, 3, 4, 8, 5, 6, 7, 'B', 2), (1, 3, 4, 8, 'B', 6, 7, 5, 2), (1, 3, 4, 8, 6, 'B', 7, 5, 2), (1, 3, 4, 8, 6, 2, 7, 5, 'B'), (1, 3, 4, 8, 6, 2, 7, 'B', 5), (1, 3, 4, 8, 'B', 2, 7, 6, 5), (1, 3, 4, 8, 2, 'B', 7, 6, 5), (1, 3, 'B', 8, 2, 4, 7, 6, 5), (1, 'B', 3, 8, 2, 4, 7, 6, 5), (1, 2, 3, 8, 'B', 4, 7, 6, 5)]


In [208]:
a_star_greater_challenge_manhattan_path = a_star_search(greater_challenge, lambda x: greater_challenge.manhattan_distance(x) + x.path_cost).path()
print(a_star_greater_challenge_manhattan_path)

[(5, 1, 4, 7, 'B', 6, 3, 8, 2), (5, 1, 4, 'B', 7, 6, 3, 8, 2), (5, 1, 4, 3, 7, 6, 'B', 8, 2), (5, 1, 4, 3, 7, 6, 8, 'B', 2), (5, 1, 4, 3, 'B', 6, 8, 7, 2), (5, 1, 4, 'B', 3, 6, 8, 7, 2), ('B', 1, 4, 5, 3, 6, 8, 7, 2), (1, 'B', 4, 5, 3, 6, 8, 7, 2), (1, 3, 4, 5, 'B', 6, 8, 7, 2), (1, 3, 4, 'B', 5, 6, 8, 7, 2), (1, 3, 4, 8, 5, 6, 'B', 7, 2), (1, 3, 4, 8, 5, 6, 7, 'B', 2), (1, 3, 4, 8, 5, 6, 7, 2, 'B'), (1, 3, 4, 8, 5, 'B', 7, 2, 6), (1, 3, 4, 8, 'B', 5, 7, 2, 6), (1, 3, 4, 8, 2, 5, 7, 'B', 6), (1, 3, 4, 8, 2, 5, 7, 6, 'B'), (1, 3, 4, 8, 2, 'B', 7, 6, 5), (1, 3, 'B', 8, 2, 4, 7, 6, 5), (1, 'B', 3, 8, 2, 4, 7, 6, 5), (1, 2, 3, 8, 'B', 4, 7, 6, 5)]


In [209]:
a_star_greater_challenge_permutation_inversions_path = a_star_search(greater_challenge, lambda x: greater_challenge.permutation_inversions(x) + x.path_cost).path()
print(a_star_greater_challenge_permutation_inversions_path)

[(5, 1, 4, 7, 'B', 6, 3, 8, 2), (5, 1, 4, 'B', 7, 6, 3, 8, 2), (5, 1, 4, 3, 7, 6, 'B', 8, 2), (5, 1, 4, 3, 7, 6, 8, 'B', 2), (5, 1, 4, 3, 'B', 6, 8, 7, 2), (5, 1, 4, 'B', 3, 6, 8, 7, 2), ('B', 1, 4, 5, 3, 6, 8, 7, 2), (1, 'B', 4, 5, 3, 6, 8, 7, 2), (1, 3, 4, 5, 'B', 6, 8, 7, 2), (1, 3, 4, 'B', 5, 6, 8, 7, 2), (1, 3, 4, 8, 5, 6, 'B', 7, 2), (1, 3, 4, 8, 5, 6, 7, 'B', 2), (1, 3, 4, 8, 'B', 6, 7, 5, 2), (1, 3, 4, 8, 6, 'B', 7, 5, 2), (1, 3, 4, 8, 6, 2, 7, 5, 'B'), (1, 3, 4, 8, 6, 2, 7, 'B', 5), (1, 3, 4, 8, 'B', 2, 7, 6, 5), (1, 3, 4, 8, 2, 'B', 7, 6, 5), (1, 3, 'B', 8, 2, 4, 7, 6, 5), (1, 'B', 3, 8, 2, 4, 7, 6, 5), (1, 2, 3, 8, 'B', 4, 7, 6, 5)]


In [210]:
a_star_greater_challenge_inadmissible_heuristic_path = a_star_search(greater_challenge, lambda x: greater_challenge.inadmissible_heuristic(x) + x.path_cost).path()
print(a_star_greater_challenge_inadmissible_heuristic_path)

[(5, 1, 4, 7, 'B', 6, 3, 8, 2), (5, 1, 4, 7, 8, 6, 3, 'B', 2), (5, 1, 4, 7, 8, 6, 'B', 3, 2), (5, 1, 4, 'B', 8, 6, 7, 3, 2), ('B', 1, 4, 5, 8, 6, 7, 3, 2), (1, 'B', 4, 5, 8, 6, 7, 3, 2), (1, 8, 4, 5, 'B', 6, 7, 3, 2), (1, 8, 4, 5, 3, 6, 7, 'B', 2), (1, 8, 4, 5, 3, 6, 7, 2, 'B'), (1, 8, 4, 5, 3, 'B', 7, 2, 6), (1, 8, 'B', 5, 3, 4, 7, 2, 6), (1, 'B', 8, 5, 3, 4, 7, 2, 6), (1, 3, 8, 5, 'B', 4, 7, 2, 6), (1, 3, 8, 5, 2, 4, 7, 'B', 6), (1, 3, 8, 5, 2, 4, 7, 6, 'B'), (1, 3, 8, 5, 2, 'B', 7, 6, 4), (1, 3, 'B', 5, 2, 8, 7, 6, 4), (1, 'B', 3, 5, 2, 8, 7, 6, 4), (1, 2, 3, 5, 'B', 8, 7, 6, 4), (1, 2, 3, 'B', 5, 8, 7, 6, 4), (1, 2, 3, 7, 5, 8, 'B', 6, 4), (1, 2, 3, 7, 5, 8, 6, 'B', 4), (1, 2, 3, 7, 'B', 8, 6, 5, 4), (1, 2, 3, 7, 8, 'B', 6, 5, 4), (1, 2, 3, 7, 8, 4, 6, 5, 'B'), (1, 2, 3, 7, 8, 4, 6, 'B', 5), (1, 2, 3, 7, 8, 4, 'B', 6, 5), (1, 2, 3, 'B', 8, 4, 7, 6, 5), (1, 2, 3, 8, 'B', 4, 7, 6, 5)]


## Measuring the lenghts of the search paths


### EightPuzzle((2, 8, 3, 1, 6, 4, 7, 'B', 5))

In [211]:
challenge_bf_path_lengths = [len(bf_challenge_hamming_path), len(bf_challenge_manhattan_path), len(bf_challenge_permutation_inversions_path), len(bf_challenge_inadmissible_heuristic_path)]
print(challenge_bf_path_lengths)

[10, 6, 6, 6]


In [212]:
challenge_a_star_path_lengths = [len(a_star_challenge_hamming_path), len(a_star_challenge_manhattan_path), len(a_star_challenge_permutation_inversions_path), len(a_star_challenge_inadmissible_heuristic_path)]
print(challenge_a_star_path_lengths)

[6, 6, 6, 6]


### EightPuzzle((5, 1, 4, 7, 'B', 6, 3, 8, 2))

In [213]:
greater_challenge_bf_path_lengths = [len(bf_greater_challenge_hamming_path), len(bf_greater_challenge_manhattan_path), len(bf_greater_challenge_permutation_inversions_path), len(bf_greater_challenge_inadmissible_heuristic_path)]
print(greater_challenge_bf_path_lengths)

[117, 79, 141, 57]


In [214]:
greater_challenge_a_star_path_lengths = [len(a_star_greater_challenge_hamming_path), len(a_star_greater_challenge_manhattan_path), len(a_star_greater_challenge_permutation_inversions_path), len(a_star_greater_challenge_inadmissible_heuristic_path)]
print(greater_challenge_a_star_path_lengths)

[21, 21, 21, 29]


In [232]:
# Some EightPuzzle problems for testing

p1 = EightPuzzle(('B', 2, 3, 1, 4, 5, 8, 7, 6))
p2 = EightPuzzle((1, 4, 2, 'B', 8, 3, 7, 6, 5))
p3 = EightPuzzle((2, 8, 1, 'B', 4, 3, 7, 6, 5))

In [233]:
p1_bf_path_lengths = [len(best_first_search(p1, lambda x: p1.hamming_distance(x)).path()), len(best_first_search(p1, lambda x: p1.manhattan_distance(x)).path()), len(best_first_search(p1, lambda x: p1.permutation_inversions(x)).path()), len(best_first_search(p1, lambda x: p1.inadmissible_heuristic(x)).path())]
print(p1_bf_path_lengths)

[7, 7, 7, 7]


In [234]:
p1_a_star_path_lengths = [len(a_star_search(p1, lambda x: p1.hamming_distance(x) + x.path_cost).path()), len(a_star_search(p1, lambda x: p1.manhattan_distance(x) + x.path_cost).path()), len(a_star_search(p1, lambda x: p1.permutation_inversions(x) + x.path_cost).path()), len(a_star_search(p1, lambda x: p1.inadmissible_heuristic(x) + x.path_cost).path())]
print(p1_a_star_path_lengths)

[7, 7, 7, 7]


In [235]:
p2_bf_path_lengths = [len(best_first_search(p2, lambda x: p2.hamming_distance(x)).path()), len(best_first_search(p2, lambda x: p2.manhattan_distance(x)).path()), len(best_first_search(p2, lambda x: p2.permutation_inversions(x)).path()), len(best_first_search(p2, lambda x: p2.inadmissible_heuristic(x)).path())]
print(p2_bf_path_lengths)

[10, 6, 6, 6]


In [236]:
p2_a_star_path_lengths = [len(a_star_search(p2, lambda x: p2.hamming_distance(x) + x.path_cost).path()), len(a_star_search(p2, lambda x: p2.manhattan_distance(x) + x.path_cost).path()), len(a_star_search(p2, lambda x: p2.permutation_inversions(x) + x.path_cost).path()), len(a_star_search(p2, lambda x: p2.inadmissible_heuristic(x) + x.path_cost).path())]
print(p2_a_star_path_lengths)

[6, 6, 6, 6]


In [237]:
p3_bf_path_lengths = [len(best_first_search(p3, lambda x: p3.hamming_distance(x)).path()), len(best_first_search(p3, lambda x: p3.manhattan_distance(x)).path()), len(best_first_search(p3, lambda x: p3.permutation_inversions(x)).path()), len(best_first_search(p3, lambda x: p3.inadmissible_heuristic(x)).path())]
print(p3_bf_path_lengths)

[44, 42, 46, 24]


In [238]:
p3_a_star_path_lengths = [len(a_star_search(p3, lambda x: p3.hamming_distance(x) + x.path_cost).path()), len(a_star_search(p3, lambda x: p3.manhattan_distance(x) + x.path_cost).path()), len(a_star_search(p3, lambda x: p3.permutation_inversions(x) + x.path_cost).path()), len(a_star_search(p3, lambda x: p3.inadmissible_heuristic(x) + x.path_cost).path())]
print(p3_a_star_path_lengths)

[10, 10, 10, 10]
